In [1]:
import random
from collections import Counter
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from tqdm.notebook import tqdm, trange
import time 

sns.color_palette('pastel')

[(0.6313725490196078, 0.788235294117647, 0.9568627450980393),
 (1.0, 0.7058823529411765, 0.5098039215686274),
 (0.5529411764705883, 0.8980392156862745, 0.6313725490196078),
 (1.0, 0.6235294117647059, 0.6078431372549019),
 (0.8156862745098039, 0.7333333333333333, 1.0),
 (0.8705882352941177, 0.7333333333333333, 0.6078431372549019),
 (0.9803921568627451, 0.6901960784313725, 0.8941176470588236),
 (0.8117647058823529, 0.8117647058823529, 0.8117647058823529),
 (1.0, 0.996078431372549, 0.6392156862745098),
 (0.7254901960784313, 0.9490196078431372, 0.9411764705882353)]

In [2]:
# Number of agents
N = 100

# Cost
c = 0.1

# Benefit
b = 1

minimum_strategy = -5
maximum_strategy = 6

minimum_image_score = -5
maximum_image_score = 5

In [3]:
class Agent:
    def __init__(self, strategy):
        self.image_score = 0
        self.payoff = 0
        self.strategy = strategy

    def increase_image_score(self):
        temp_score = self.image_score + 1
        if temp_score > maximum_image_score:
            self.image_score = maximum_image_score
        else:
            self.image_score = temp_score

    def decrease_image_score(self):
        temp_score = self.image_score - 1
        if temp_score < minimum_image_score:
            self.image_score = minimum_image_score
        else:
            self.image_score = temp_score

In [4]:
def play_round(m=60, iterations=150):
    agents = []
    coalitions = []
    #network = nx.barabasi_albert_graph(N, 3)
    network = nx.random_graphs.watts_strogatz_graph(N, 5, 0.1)

    final_strategies = []
    number_of_coalitions_per_round = []

    for i in range(N):
        agent = Agent(random.randint(minimum_strategy, maximum_strategy))
        agents.append(agent)
        network.nodes[i]['agent'] = agent

    for iteration in range(iterations):

        for pair_number in range(m):
            a_i, a_j = find_players(agents)
            change_payoff_score(a_i, a_j, network, coalitions)

        for i in range(N):
            network.nodes[i]['oldValue'] = network.nodes[i]['agent'].strategy
        
        for a_i in agents:
            # Change Strategy
            change_strategy(a_i, network)

        for a_i in agents:
            # Reset Image Score and Payoff
            reset_image_score_and_payoff(a_i)


    for i in range(N):
        final_strategies.append(network.nodes[i]['agent'].strategy)
    #print(Counter(final_strategies))
    return final_strategies

In [5]:
def change_strategy(a_i, network):
    neighbors_payoffs = []
    neighbors_strategies = []

    for neighbor in get_neighbors(a_i, network):
        neighbors_payoffs.append(network.nodes[neighbor]['agent'].payoff)
        neighbors_strategies.append(network.nodes[neighbor]['oldValue'])

    neighbors_payoffs.append(a_i.payoff)
    neighbors_strategies.append(a_i.strategy)

    max_payoff = max(neighbors_payoffs)

    for i in range(len(neighbors_payoffs)):
        if neighbors_payoffs[i] == max_payoff:
            a_i.strategy = neighbors_strategies[i]
            break


def reset_image_score_and_payoff(a_i):
    a_i.image_score = 0
    a_i.payoff = 0


def change_payoff_score(a_i, a_j, network, coalitions):
    a_j_score = 0
    if is_neighbor(a_i, a_j, network):
        a_j_score = a_j.image_score

    if a_i.strategy <= a_j_score:
        a_i.payoff -= c
        a_j.payoff += b
        a_i.increase_image_score()
    else:
        a_i.decrease_image_score()


def get_neighbors(a_i, network):
    for node, node_data in network.nodes(data=True):
        if network.nodes[node]['agent'] == a_i:
            return network.neighbors(node)


def is_neighbor(a_i, a_j, network):
    for node, node_data in network.nodes(data=True):
        if network.nodes[node]['agent'] == a_i:
            for neighbor in network.neighbors(node):
                if network.nodes[neighbor]['agent'] == a_j:
                    return True
            break
    return False


def find_players(agents):
    a_i, a_j = random.sample(agents, 2)
    while a_i == a_j:
        a_i, a_j = random.sample(agents, 2)
    return a_i, a_j

In [6]:
strategies = []
for i in trange(1000):
    values = play_round(m=200, iterations= 600)
    for value in values:
        strategies.append(value)
    


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import altair as alt
import pandas as pd

percentages = []
percentages.append(len([strat for strat in strategies if strat == 1])/len(strategies))
percentages.append(len([strat for strat in strategies if strat == 2])/len(strategies))
percentages.append(len([strat for strat in strategies if strat == 3])/len(strategies))
percentages.append(len([strat for strat in strategies if strat == 4])/len(strategies))
percentages.append(len([strat for strat in strategies if strat == 5])/len(strategies))
percentages.append(len([strat for strat in strategies if strat == 6])/len(strategies))


data = pd.DataFrame({'Strategy': np.array([1,2,3,4,5,6]),'Frequency':np.array(percentages),  'color':['#F2994A' for c in range(len(percentages))]})


bar = alt.Chart(data).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(x=alt.X('x', scale=alt.Scale(domain=(minimum_strategy, maximum_strategy))), y='Frequency', ).encode(
    x=alt.X('Strategy', scale=alt.Scale(domain=(minimum_strategy, maximum_strategy))),
    color=alt.Color('color', scale=None)
)


rule = alt.Chart(data).mark_rule(color='red').encode(
    y='mean(Frequency):Q'
)


(bar + rule)



In [ ]:

alt.Chart(data).mark_bar().transform_fold(
    fold=['Strategy', 'Frequency'], 
    as_=['x', 'y']
).encode(
    x='Strategy',
    y='Frequency',
    color=alt.Color('Strategy:N', legend=alt.Legend(columns=2))
)